_Clone Before Using to play with Features / Hyperparameters_

# Project

### Import & load Data

In [0]:
pip install kaggle

In [0]:
### Everyone has to do this himself, with his own kaggle.json -> get it from kaggle as api token
import os

# Folder containing kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = "/Workspace/Users/20250355@novaims.unl.pt"

# Optional: test
!echo $KAGGLE_CONFIG_DIR

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import seaborn as sns

from sklearn.feature_selection import VarianceThreshold, RFE, chi2
from scipy.stats import spearmanr
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, FunctionTransformer


from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import Ridge, Lasso, LassoCV, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, StackingRegressor
from sklearn.svm import SVR

from data_cleaning import clean_car_dataframe

df_cars_train = pd.read_csv("train.csv")
df_cars_test = pd.read_csv("test.csv")

### Explorative Data Analysis (EDA)

TASK I (3 Points): Descriptive Statistics, Inconsistency Check, Visual Data Explorance, Extraction of Relevant Insights, Multivariate Relationships  => Explain

In [0]:
# Overview of structure and data types
df_cars_train.info()

# Check for duplicates
print(f"\nDuplicate rows: {df_cars_train.duplicated().sum()}") # 0

# Check for duplicate carID
print(f"\nDuplicate carID: {df_cars_train['carID'].duplicated().sum()}") # 0

# Check for null values in column CarID
print(f"\nNull values in column carID: {df_cars_train['carID'].isnull().sum()}") # 0

# Findings:
#   - missing values in every column except carID
#   - year, mpg, previousOwners, hasDamage as float seems weird
#   - no duplicates in all columns, no null values in carID

In [0]:
# print exact unique values of df_cars_train before doing describe
for col in df_cars_train.columns:
    print(col, df_cars_train[col].unique())

# describe all
df_cars_train.describe(include="all").T

In [0]:
# Univariate Exploration of relevant Columns

sns.set(style="whitegrid")

num_cols = ['price','mileage','tax','mpg','engineSize','paintQuality%','previousOwners','year']

# colors
hist_color = '#1f77b4'   # dark blue
box_color = '#ff7f0e'    # warm orange

# figure: 2 features per row (4 plots per row = hist + boxplot per feature)
n_features = len(num_cols)
n_rows = int(np.ceil(n_features / 2))
fig, axes = plt.subplots(n_rows, 4, figsize=(16, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    # histogram
    sns.histplot(df_cars_train[col], bins=25, color=hist_color, kde=False, ax=axes[i*2])
    axes[i*2].set_xlabel('')
    axes[i*2].set_ylabel('')
    axes[i*2].set_title(f'{col}', fontsize=11, pad=12)

    # boxplot (vertical)
    sns.boxplot(y=df_cars_train[col], ax=axes[i*2 + 1], color=box_color)
    axes[i*2 + 1].set_xlabel('')
    axes[i*2 + 1].set_ylabel('')
    axes[i*2 + 1].set_title(f'{col}', fontsize=11, pad=12)

# hide any unused axes
for j in range(i*2 + 2, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('univariate distributions of numerical features', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

%md
##### Findings after Descriptive Statistics and Inconsistency Check

carID:  
- sequential numeric identifier, ranges from **0–75,972**  
- no duplicates expected, used only as index/key  

brand:  
- **72 unique brands** with severe spelling and capitalization inconsistencies (**Ford**, **ford**, **FOR**, **ord**, **For**, etc.)  
- contains partial or truncated names (**w**, **MW**, **Ope**, **Mercede**) → heavy cleaning required  
- **Ford dominates (~15k entries)**, followed by **BMW**, **VW**, **Mercedes**, **Toyota**, **Audi**, **Skoda**, **Hyundai**  
- strong class imbalance → use frequency encoding or median price per brand later  

model:  
- **735 unique entries** with inconsistent formatting, spacing, capitalization, and partial strings  
- duplicates of same model under variations (e.g. *“focus”*, *“ FOCUS”*, *“ Focu”*, *“Focus”*)  
- many small typos, truncated or malformed entries (*“Focu”*, *“EcoSpor”*, *“Gol”*, *“Yeti Outdoo”*, etc.)  
- heavy normalization needed → strip whitespace, lowercase, and fix common truncations  
- dominated by popular models like **Ford Focus**, **VW Golf**, **Vauxhall Astra/Corsa**, **Skoda Octavia**  

year:  
- values range from **1970–2024**, mean ≈ **2017**  
- years after **2020** are unrealistic (future registration) → invalid entries  
- decimals in year (e.g. **2023.367**) → data corruption, round to nearest int  
- older outliers before **2000** rare, likely classic or miscoded entries  
- can derive **age = 2020 - year** for modeling  

transmission:  
- **40 distinct entries**, mostly spelling variants of *manual*, *automatic*, *semi-auto*  
- common corruptions: *manua*, *anual*, *semi-aut*, *utomatic*, *nknow*, etc.  
- some leading/trailing spaces (*' manual '*, *' Manual '*)  
- categories should be reduced to clean labels: **manual**, **automatic**, **semi-auto**, **unknown**  

mileage:  
- range **–58,540 → 323,000**, mean ≈ **23k** → negatives invalid  
- missing values around **1.5k**  
- strong right skew, some extreme outliers >**250k miles**  
- negative or zero values should be filtered or replaced with abs()  

tax:  
- range **–91 → 580**, mean ≈ **120** → invalid negatives present  
- normal values cluster around **125–145**  
- decimals and small negatives appear due to calculation/entry errors  
- likely strong right skew → a few cars taxed over **500**  
- needs capping and replacement for negatives  

mpg:  
- range **–43 → 470**, mean ≈ **55** → negative and extreme outliers exist  
- typical real range **30–70 mpg**, but some values like **470** unrealistic  
- invalid entries indicate unit mix-up or input noise  
- expected inverse relation with **engine size** and **price**  

engineSize:  
- range **–0.1 → 6.6L**, mean ≈ **1.66L**, std ≈ **0.57**  
- several decimals and negative/zero values → invalid  
- expected valid range **0.6–6.0L**  
- most cars between **1.2–2.0L** → compact to mid-size engines  
- positive correlation with **price** and **tax**  

paintQuality%:  
- range **1.6 → 125.6%**, mean ≈ **64.6%**  
- values above **100%** unrealistic → scaling error  
- some extremely low values (**≈1–3%**) indicate outliers or noise  
- most cars between **50–80%** → average paint quality  

previousOwners:  
- range **–2.3 → 6.26**, mean ≈ **2**  
- negative values invalid → likely placeholder or encoding issue  
- most between **0–3** → typical secondhand ownership distribution  
- outliers >**6** likely data entry errors or mis-scaling  
- likely negative correlation with **price**  

hasDamage:  
- only values are **0 and NaN** → no variation  
- unclear if NaN means damaged → convert to int and verify meaning  
- likely nonfunctional feature → **drop**  

price (target):  
- range **£450–£159,999**, mean ≈ **£16.9k**, median ≈ **£14.7k** → right-skewed  
- typical cars priced **£10k–£21k**, few luxury outliers inflate mean  
- consistent integer values, no missing or obvious anomalies  
- strong dependence expected on **mileage**, **year**, **engine size**, and **brand**
</div>


In [0]:
# pearson and spearman correlation

num_cols = ['price','mileage','tax','mpg','engineSize','paintQuality%','previousOwners','year']

corr_pearson = df_cars_train[num_cols].corr(method='pearson', numeric_only=True).round(2)
corr_spearman = df_cars_train[num_cols].corr(method='spearman', numeric_only=True).round(2)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
mask = np.triu(np.ones_like(corr_spearman, dtype=bool))

sns.heatmap(corr_pearson, mask=mask, annot=True, cmap='coolwarm', center=0, fmt='.2f',
            linewidths=0.5, ax=axes[0], cbar=False)
axes[0].set_title('pearson correlation (linear)')

sns.heatmap(corr_spearman, mask=mask, annot=True, cmap='coolwarm', center=0, fmt='.2f',
            linewidths=0.5, ax=axes[1], cbar_kws={'label': 'correlation strength'})
axes[1].set_title('spearman correlation (monotonic)')

plt.suptitle('comparison of pearson vs spearman correlation', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()


##### Why we decided to use Spearman correlation:

- the numeric variables (price, mileage, mpg, tax, engineSize) are **non-normally distributed and contain outliers**, causing linear measures like pearson to distort correlation strength.  
- **spearman** evaluates *monotonic* relationships based on rank order rather than exact linearity, making it **robust to skewness and outliers**.  
- this allows us to correctly capture the direction and magnitude of real-world trends, such as price decreasing with mileage, even if not perfectly linear.  
- after cleaning and scaling the data, we will revisit **pearson correlation** for linear modeling checks, but **spearman is more appropriate for initial EDA**.

##### Findings after correlation:

- price shows **strong positive correlation with year (0.60)** → newer cars are priced higher  
- price is **positively correlated with engineSize (0.56)** → larger engines increase car value  
- price is **moderately negatively correlated with mileage (–0.51)** → more driven cars lose value  
- mpg correlates **negatively with price (–0.39)** → efficient cars are typically smaller and cheaper  
- tax has a **moderate positive correlation (0.31)** with price → more expensive cars often have higher taxes  
- year and mileage have a **very strong negative correlation (–0.78)** → newer cars have lower mileage  
- mpg and tax are **strongly negatively correlated (–0.55)** → efficient cars usually taxed less  
- engineSize and mpg **correlate negatively (–0.20) → larger engines are less fuel-efficient  
- paintQuality% and previousOwners show **near-zero correlations** with all other variables → low predictive relevance  
- overall, **price mainly depends on year, engineSize, mileage, and mpg**, which align with intuitive market behavior


In [0]:
# Multivariate Analysis



%md
##### Findings after Multivariate Analysis


### Data Cleaning, Feature Engineering, Split & Preprocessing

Task II (5 Points): Clean and preprocess the dataset. 
- Missing Value handling, Outlier preprocessing + justify decisions -> in data_cleaning.py
- Review current features and create extra features if needed + explain
- Deal with categorical variables -> One-Hot-Encoding 
- Perform data scaling, explain reasoning

In [0]:
# Outlier Preprocessing happens here
df_cars_train = clean_car_dataframe(df_cars_train)
df_cars_test = clean_car_dataframe(df_cars_test)


# Safety Check: print all unique values of all columns of df_cars_train // df_cars_test to see if data cleaning worked and if there are still weird values
for col in df_cars_train.columns:
    print(col, df_cars_train[col].unique())
print("X"*150)
for col in df_cars_test.columns:
    print(col, df_cars_test[col].unique())

In [0]:
# Feature Engineering and Explaination

# add column age: models can easier interpret linear numerical features
df_cars_train['age'] = 2025 - df_cars_train['year']
df_cars_test['age'] = 2025 - df_cars_test['year']

# miles per year: normalizes the total mileage by how old the car is
df_cars_train['miles_per_year'] = df_cars_train['mileage'] / df_cars_train['age'].replace({0: np.nan})
df_cars_train['miles_per_year'] = df_cars_train['miles_per_year'].fillna(df_cars_train['mileage'])

df_cars_test['miles_per_year'] = df_cars_test['mileage'] / df_cars_test['age'].replace({0: np.nan})
df_cars_test['miles_per_year'] = df_cars_test['miles_per_year'].fillna(df_cars_test['mileage'])

# model frequency: some models are more common, which means they can be cheaper (supply) or retain their values better (demand). freq shows their popularity
model_freq = df_cars_train['model'].value_counts(normalize=True).to_dict()
df_cars_train['model_freq'] = df_cars_train['model'].map(model_freq)

model_freq = df_cars_test['model'].value_counts(normalize=True).to_dict()
df_cars_test['model_freq'] = df_cars_test['model'].map(model_freq)

# brand median price (only train): shows brand positioning (e.g. BMW > KIA)
brand_median_price = df_cars_train.groupby('Brand')['price'].median()
df_cars_train['brand_med_price'] = df_cars_train['Brand'].map(brand_median_price)

# model median price (only train): shows model positioning (e.g. 3er > 1er)
model_med_price = df_cars_train.groupby('model')['price'].median()
df_cars_train['model_med_price'] = df_cars_train['model'].map(model_med_price)

In [0]:
X = df_cars_train.drop(columns='price')
y = df_cars_train['price']

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.3, 
                                                  random_state = 42, 
                                                  #stratify = y,    # if y, class proportions get preserved between train and test sets
                                                  shuffle = True)

In [0]:
# Define which columns are numeric vs categorical (mileage not in here because skewed - log)
numeric_features = ["age", "tax", "mpg", "engineSize", "paintQuality", "previousOwners"]
log_features = ["mileage", "miles_per_year", "model_freq", "brand_med_price", "model_med_price"] # could try to test previousOwners, age here, tax, mpg didnt work
categorical_features = ["Brand", "model", "transmission", "fuelType"]


# left out columns: year (age is better), hasDamage (unsure what the two values 0 and NaN mean)


log_transformer = Pipeline([
    # Handling of missing numerical values with sklearn SimpleImputer (mean)
    ("imputer", SimpleImputer(strategy="mean")),
    # Data Scaling with sklearn FunctionTransformer (for log) and StandardScaler
    ("log", FunctionTransformer(np.log1p, validate=False)),
    ("scaler", StandardScaler())
])

numeric_transformer = Pipeline([
    # Handling of missing numerical values with sklearn SimpleImputer (mean)
    ("imputer", SimpleImputer(strategy="mean")),
    # Data Scaling with sklearn StandardScaler
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    # Handling of missing categorical values with sklearn SimpleImputer (Unknown)
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    # Deal with Categorical Variables with sklearn OneHotEncoder:
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False)) 
])

preprocessor = ColumnTransformer([
    # Apply the preprocessing steps to the data
    ("mileage", log_transformer, ["mileage"]), # log because mileage is skewed
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
], remainder="drop")


# Fit preprocessor on training data - avoid data leakage
preprocessor.fit(X_train)




# ColumnTransformer lets you apply different transformations to different feature subsets:
#       > Numeric → impute mean + scale
#       > Categorical → impute "Unknown" + OneHotEncode
#       > Mileage → impute log-transform
#   This is key, because numeric and categorical data need different math, you can't scale strings or one-hot encode continuous numbers.


# Pipeline bundles preprocessing + model training:
#     > Cross-validation applies preprocessing inside each fold (no data leakage).
#     > The final model object (after .fit()) knows exactly how to preprocess new data.
#     > When saving the pipeline with joblib, everything (scaler, encoder, model) is saved together.

### Feature Selection

Task III (3 Points): Define and Implement a clear and unambiguous strategy for feature selection. Use the methods discussed in the course. Present and justify your final selection 

Model independent Filter Methods:
- Remove constant numerical variables with VarianceThreshold (manual)
- Check highly correlated numerical variables and keep one with Spearman (manual)
- Remove unindependent categorical variables with Chi2

Model dependent Wrapper Methods:
- RFE LR / RFE SVR for linear Models: Ridge, Lasso, ElasticNet, SVM
- Feature Importance for tree Models: DecisionTrees, RandomForest, GradientBoosting => trees are unsensitive to irrelevant features but doing feature importance and remove some can reduce dimensionality
- L1 Regularization for Neural Networks: MLP


In [0]:
X_train_proc = preprocessor.transform(X_train)

feature_names_all = []
for name, trans, cols in preprocessor.transformers_:
    if name != 'remainder':
        if hasattr(trans, 'get_feature_names_out'):
            # for categorical OHE
            try:
                feature_names_all.extend(trans.get_feature_names_out(cols))
            except:
                feature_names_all.extend(cols)
        else:
            feature_names_all.extend(cols)

X_df = pd.DataFrame(X_train_proc, columns=feature_names_all)


# Variance Threshold
vt = VarianceThreshold(threshold=0.0)
vt.fit(X_df)
vt_deselect = [f for f, keep in zip(feature_names_all, vt.get_support()) if not keep]
print("Features to deselect according to VarianceThreshold:", vt_deselect)


# Spearman correlation (numeric + log only)
numeric_log = numeric_features + log_features
spearman_deselect = []
for f in numeric_log:
    if f in X_df.columns:
        corr, _ = spearmanr(X_df[f], y_train)
        if abs(corr) <= 0.05:
            spearman_deselect.append(f)
print("Features to deselect according to Spearman correlation:", spearman_deselect)


# Chi2 (categorical only, must be non-negative)
cat_cols = [c for c in X_df.columns if c not in numeric_log]
X_cat = X_df[cat_cols].astype(float)
chi2_vals, _ = chi2(X_cat, y_train)
chi2_deselect = [f for f, val in zip(cat_cols, chi2_vals) if val <= 0]
print("Features to deselect according to Chi²:", chi2_deselect)


In [0]:
# Numeric/log features for linear models
linear_numeric_features = [f for f in numeric_features + log_features if f not in spearman_deselect]

preprocessor_linear = ColumnTransformer([
    ("num", numeric_transformer, linear_numeric_features),
    ("cat", categorical_transformer, categorical_features)
], remainder="drop")

# => use preprocessor_linear for linear model setup; since tree models are indifferent to irrelevant features

### Models Setup and Baselining (with SKLEARN)

TASK IV (4 Points): Build a simple model and assess the performance
- Identify the type of problem and select the relevant algorithms
- Select one Model Assessment Strategy to use throughout your work. Which metrics are you using to evaluate your model and why?


In [0]:
# Following metrics are used for model evaluation:
#
#   MAE: Average absolute deviation between predicted and true car prices, easy to interpret, kaggle competition uses same metric
#   RMSE: Root mean squared error, helps to see if large errors on same values were made, therefore sensitive to outliers
#   R2: Proportion of variance explained by the model, 1 = perfect, 0 = same as predicting mean, < 0 = worse than mean baseline
#
# Because our task is a regression problem and we are predicting a continuous variable (car price)

def print_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"MAE: {mae:.4f} | RMSE: {rmse:.4f} | R2: {r2:.4f}")
    return

In [0]:
### LINEAR MODEL
# ElasticNet
elastic_pipe = Pipeline([
    ("preprocess", preprocessor_linear),
    ("model", ElasticNet(
        max_iter=20000,
        selection="random",
        warm_start=False,  # set True only if iteratively tuning manually
        random_state=42
    ))
])


### TREE MODELS
# GradientBoostingRegressor: baseline, has to beat ElasticNet, if not something’s wrong with data preprocessing, not the model.
gbr_pipe = Pipeline([
    ("preprocess", preprocessor), 
    ("model", GradientBoostingRegressor(loss='absolute_error'))
])

# HistGradientBoostingRegressor: modern and very fast, handles missing values natively (no imputation needed!). often matches or beats XGBoost/LightGBM 
hgb_pipe = Pipeline([
    ("preprocess", preprocessor),
    ("model", HistGradientBoostingRegressor(
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=20,
        l2_regularization=0.5,  # regularize slightly to prevent overfit, > 0.5 does not seem to work
    ))
])


# RandomForestRegressor: excellent general baseline ensemble, handles non-linearities well, doesn’t overfit easily but can be slow for large data
rf_pipe = Pipeline([
    ("preprocess", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=300, 
        max_depth=None, 
        n_jobs=-1, 
        random_state=42
    ))
])

# ExtraTreesRegressor: similar to RandomForest but with more randomization => often better generalization
et_pipe = Pipeline([
    ("preprocess", preprocessor),
    ("model", ExtraTreesRegressor(
        n_estimators=300, 
        max_depth=None, 
        n_jobs=-1, 
        random_state=42
    ))
])


### KERNEL BASED MODEL
# SVR: powerful, but slow on large data, sensitive to scaling => already handled in preprocessing
svr_pipe = Pipeline([
    ("preprocess", preprocessor),
    ("model", SVR(C=10, epsilon=0.2, kernel="rbf"))
])


### ENSEMBLE META MODEL
# StackingRegressor: stacks/blends multiple models => typically gives a small but consistent boost in leaderboard competitions
stack_pipe = StackingRegressor(
    estimators=[
        ("el", elastic_pipe),
        ("hgb", hgb_pipe),
    ],
    passthrough=False   # True can sometimes help but increases overfitting risk
)


In [0]:
mean_pred = y_train.mean()
median_pred = y_train.median()

print("baseline mean predictor: ")
print_metrics(y_val, [mean_pred]*len(y_val))
# MAE: 6976.3626 | RMSE: 92839550.2849 | R2: -0.0000

print("-"*150)

print("baseline median predictor: ") 
print_metrics(y_val, [median_pred]*len(y_val))
# MAE: 6751.1604 | RMSE: 97557866.6363 | R2: -0.0508

### Hyperparameter Tuning

In [0]:
elastic_param_grid = {
    "model__alpha": [0.001, 0.01, 0.05, 0.1, 0.5],
    "model__l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]     
}

elastic_grid = GridSearchCV(
    elastic_pipe, 
    param_grid=elastic_param_grid,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1,
    verbose=1
)

elastic_grid.fit(X_train, y_train)
elastic_best = elastic_grid.best_estimator_
elastic_val_pred = elastic_best.predict(X_val)


print("ElasticNet Results: ")
print_metrics(y_val, elastic_val_pred)
print("Best ElasticNet params:", elastic_grid.best_params_)

# MAE: 2540.2806 | RMSE: 16642911.3643 | R2: 0.8207
# Best ElasticNet params: {'model__alpha': 0.001, 'model__l1_ratio': 0.9}

#### HT: tree Models

In [0]:
rf_param_grid = {
    "model__max_depth": [None, 10, 20],
    "model__max_features": ["sqrt", "log2"]
}

rf_grid = GridSearchCV(
    rf_pipe, rf_param_grid,
    cv=5, scoring="r2",
    n_jobs=-1, verbose=1
)

rf_grid.fit(X_train, y_train)
rf_best = rf_grid.best_estimator_
rf_val_pred = rf_best.predict(X_val)

print("Random Forest Results: ")
print_metrics(y_val, rf_val_pred)

# 1st Try: MAE: 1431.4111 | RMSE: 5764825.2457 | R2: 0.9379 with max_depth= None, 10, 20; max_features= sqrt, log2; cv = 5

In [0]:
et_param_grid = {
    "model__max_depth": [None, 10, 20],
    "model__max_features": ["sqrt", "log2"]
}

et_grid = GridSearchCV(
    et_pipe, et_param_grid,
    cv=5, scoring="r2",
    n_jobs=-1, verbose=1
)

et_grid.fit(X_train, y_train)
et_best = et_grid.best_estimator_
et_val_pred = et_best.predict(X_val)

print("ExtraTrees Results: ")
print_metrics(y_val, et_val_pred)

# 1st Try: MAE: 1438.3138 | RMSE: 5865741.8358 | R2: 0.9368 with max_depth= None, 10, 20; max_features= sqrt, log2; cv = 5

In [0]:
gbr_param_grid = {
    "model__n_estimators": [500],
    "model__learning_rate": [0.05],
    "model__max_depth": [5], "model__loss": ['squared_error']
}

gbr_grid = GridSearchCV(
    gbr_pipe, gbr_param_grid,
    cv=5, scoring="r2",
    n_jobs=-1, verbose=1
)

gbr_grid.fit(X_train, y_train)
gbr_best = gbr_grid.best_estimator_
gbr_val_pred = gbr_best.predict(X_val)

print("GradientBoosting Results: ")
print_metrics(y_val, gbr_val_pred)

In [0]:
hgb_param_grid = {
    "model__learning_rate": [0.06], # also tried: 0.02, 0.04, 0.05, 0.1
    "model__max_leaf_nodes": [50], # also tried: 15, 25, 31, 60
    "model__min_samples_leaf": [5], # also tried: 8, 10, 15, 20
    "model__max_iter": [800] # also tried: 500, 1000
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

hgb_grid = GridSearchCV(
    estimator=hgb_pipe,
    param_grid=hgb_param_grid,
    cv=cv,
    scoring="neg_mean_absolute_error",  # optimize MAE
    n_jobs=-1,
    verbose=2
)

hgb_grid.fit(X_train, y_train)
hgb_best_1 = hgb_grid.best_estimator_

hgb_val_pred = hgb_best_1.predict(X_val)
print_metrics(y_val, hgb_val_pred)

# Best Parameters: {'model__learning_rate': 0.06, 'model__max_iter': 800, 'model__max_leaf_nodes': 50, 'model__min_samples_leaf': 5}
# MAE: 1304.7611 | RMSE: 4503446.5247 | R2: 0.9515 
# joblib.dump(hgb_best_1, "hgb_best_1.pkl")

# Save model for later use
joblib.dump(hgb_XYZ, "hgb_XYZ") # neuen namen vergeben und speichern wenn besser als hgb_best_1

In [0]:
hgb_param_grid = {
    "model__learning_rate": [0.06, 0.07, 0.08],
    "model__max_leaf_nodes": [45, 50, 55],
    "model__min_samples_leaf": [4, 5, 6],
    "model__max_iter": [700, 800, 900]
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

hgb_grid = GridSearchCV(
    estimator=hgb_pipe,
    param_grid=hgb_param_grid,
    cv=cv,
    scoring="neg_mean_absolute_error",  # optimize MAE
    n_jobs=-1,
    verbose=2
)

hgb_grid.fit(X_train, y_train)
hgb_best_2 = hgb_grid.best_estimator_

hgb_val_pred = hgb_best_2.predict(X_val)
print_metrics(y_val, hgb_val_pred)

#### HT: advanced Models

In [0]:
svr_param_grid = {
    "model__C": [1, 10, 100],
    "model__epsilon": [0.1, 0.2],
    "model__kernel": ["rbf"]
}

svr_grid = GridSearchCV(
    svr_pipe, svr_param_grid,
    cv=cv, scoring="r2", n_jobs=-1, verbose=1
)

svr_grid.fit(X_train, y_train)
svr_best = svr_grid.best_estimator_

svr_val_pred = svr_best.predict(X_val)
r2_svr, rmse_svr, mae_svr = print_metrics(y_val, svr_val_pred)

In [0]:
# stackingregressor (elasticnet + histgradientboost)

param_grid_stack = {
    "final_estimator__learning_rate": [0.03, 0.05, 0.08],
    "final_estimator__max_depth": [3, 5, 7],
    "final_estimator__min_samples_leaf": [5, 10, 20]
}

stack_grid = GridSearchCV(
    stack_pipe,
    param_grid=param_grid_stack,
    scoring="neg_mean_absolute_error",
    cv=3,            # lower CV since it’s meta-tuning
    n_jobs=-1,
    verbose=2
)

stack_grid.fit(X_train, y_train)
stack_best = stack_grid.best_estimator_

val_pred = stack_best.predict(X_val)
print("Best params:", stack_grid.best_params_)
print_metrics(y_val, val_pred)



### Model Evaluation

Extra Task (1 Point): Be in the Top 5 Groups on Kaggle

In [0]:
# load hgb_best_1 from joblib
hgb_best_1 = joblib.load("hgb_best_1.pkl")

In [0]:
# Pick best model and predict on test:
df_cars_test['price'] = hgb_best_1.predict(df_cars_test)

df_cars_test['price'].to_csv('submission.csv', index=True)


In [0]:
!kaggle competitions submit -c cars4you -f submission.csv -m "Message"

In [0]:
# Kaggle Score Check
!kaggle competitions submissions -c cars4you

### Feature Importance

In [0]:
# What/Why: Different models give different “importance” signals:
# Lasso → coefficients (linear importance); zero → removed feature.
# Tree ensembles (GBR) → feature_importances_ (importance in splits).
# For rigorous interpretation, use SHAP for consistent feature attributions across models.


# Important: yes — each model may select different features. That’s expected. Use the model type that matches your use-case:
# If you need a sparse, interpretable linear model → use Lasso.
# If you need best predictive power on tabular data → use ensemble/boosting and interpret via SHAP.

#### Iterative Loop Checklist

The loop starts after you look at baseline performance and diagnostics:
- Baseline → Check metrics on validation (and residuals).
- Inspect failures / residual plots / feature importances (did a certain brand get consistently over/under predicted?)
- Hypothesize (e.g. add interaction year * mileage, try log transform for horsepower, create age = current_year - year).
- Implement changes in pipeline (e.g. add FunctionTransformer for log(horsepower) or PolynomialFeatures on a small set).
- Re-run CV/hyperparameter search and evaluate again.
- Log results, repeat.

Note on feature selection: yes — different models will select different subsets. Typical approaches:
- Use Lasso or SelectFromModel as a filter for linear pipelines.
- Use tree-based model importances or SHAP to select features for simpler models.
- Or let the best predictive model use all features (trees are robust to redundancy).

Final notes (recommended best-practices)
- Always fit preprocessing only on training data (pipelines do this automatically if you use them inside CV).
- Start simple: mean baseline → Ridge/Lasso → tree-based. Use the simple models for interpretability and as sanity checks.
- For heavy hyperparameter searches use RandomizedSearchCV or Optuna if the space is big.
- When comparing models, report multiple metrics (R², MAE, RMSE). For price prediction MAE is often most interpretable.
- For reproducibility, store your dataset version, random seed, code, and results log.